In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import os 
from time import sleep
from multiprocessing import Queue, Manager, Process

from functions import yt_dl, pth_vids, pth_vid
from master_functions import iter0, sfm_pipe 

In [19]:
def master_iter0(args):

    """Performs iter0() and launch sfm_pipe processes with 
    the generated sets."""
    
    path_set_dir, width = iter0(*args)
    pth_sets = [os.path.join(path_set_dir, el) for el in os.listdir(path_set_dir)]
    # start new process
    for pth_set in pth_sets:
        args = (pth_set, width)
        Process(target=sfm_pipe, args=args).start()

def launching(rate, f_frms, f_ftrs, f_mtchs, sample, plylsts=[], vids=[]):
    
    for plylst in plylsts:
        for v_id in os.listdir(pth_plylst(plylst)):
            args = (pth_vid(v_id, plylst), rate,f_frms, f_ftrs, f_mtchs, sample)
            Process(target=master_iter0, args=args).start()

    for v_id in vids:
        args = (pth_vid(v_id), rate, f_frms, f_ftrs, f_mtchs, sample)
        Process(target=master_iter0, args=(args,)).start()
                  
def drain(rate, f_frms, f_ftrs, f_mtchs, sample, plylsts=[], vids=[]):
    
    print('Start Draining')
    execution = Process(target=launching, args=(rate, f_frms, f_ftrs, f_mtchs, sample, plylsts, vids))
    execution.start()
    execution.join()
    print('Finish Draining')

In [20]:
rate = 2
sample = True 
f_frms  = f_ftrs = f_mtchs = True
PLYLST = []

VIDS = os.listdir(pth_vids())

drain(rate, f_frms, f_ftrs, f_mtchs, sample,
      plylsts=PLYLST, vids=VIDS)

Start Draining


Process Process-6:2:
Traceback (most recent call last):
  File "/Users/xrubiato/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/Users/xrubiato/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-19-b49d0d5958e8>", line 6, in master_iter0
    path_set_dir, width = iter0(*args)
  File "/Users/xrubiato/DHLab/Draining-Youtube/master_functions.py", line 42, in iter0
    xtrct_frame(v_id, plylst, sample, rate)
  File "/Users/xrubiato/DHLab/Draining-Youtube/functions.py", line 189, in xtrct_frame
    make_dir(path_frames)
  File "/Users/xrubiato/DHLab/Draining-Youtube/functions.py", line 42, in make_dir
    os.mkdir(pth_dir)
FileNotFoundError: [Errno 2] No such file or directory: '/Users/xrubiato/DHLab/Draining-Youtube/videos/.DS_Store/frames'
Process Process-6:1:1:
Traceback (most recent call last):
  File "/Users/xrubiato/anaconda3/lib/python3.6/multipro

Finish Draining


In [ ]:
yt_dl('https://www.youtube.com/watch?v=b3dMHvxGIKY')

### Test 

In [9]:
def f2(): 
    sleep(3)
    print('f2')
def f1(): 
    print('start f1')
    Process(target=f2, args=()).start()
    print('finish f1')

In [12]:
p = Process(target=f1, args=())
p.start()
p.join()

start f1
finish f1
f2


## Archive 

In [ ]:
class Worker:

    def __init__(self, Q):
        self.Q = Q


    def Qiter0(self, args):
        """Performs iter0() and enqueues an sfm_pipe process with the generated sets."""
        path_set_dir, width = iter0(*args)
        pth_sets = [os.path.join(path_set_dir, el) for el in os.listdir(path_set_dir)]
        print(width)
        # Enqueue next tasks
        for pth_set in pth_sets:
            args = (pth_set, width)
            self.Q.put(('sfm_pipe', args))


    def sfm_pipe(self, args):
        sfm_pipe(*args)


    def work(self, func, args):

        if func == 'Qiter0':
            self.Qiter0(args)

        elif func == 'sfm_pipe':
            self.sfm_pipe(args)

            
def worker_exec(Q, wip_l):
    
    func, args = Q.get()
    # append to wip list 
    wip_l.append(func)
    
    # instanciate worker 
    worker = Worker(Q)
    worker.work(func, args)
    # remove from list after it finishes 
    wip_l.remove(func)

    
def drain(rate, f_frms, f_ftrs, f_mtchs, sample,
           plylsts=[], vids=[]):
    
    Q = Queue()
    manager = Manager()
    wip_l = manager.list()
    
    for plylst in plylsts:
        for v_id in os.listdir(pth_plylst(plylst)):
            
            args = (pth_vid(v_id, plylst), rate, f_frms, f_ftrs, f_mtchs, sample)
            Q.put(('Qiter0', args))

    for v_id in vids:
            
            args = (pth_vid(v_id), rate, f_frms, f_ftrs, f_mtchs, sample)
            Q.put(('Qiter0', args))
                  
    while True:
        
        if not Q.empty():
            Process(target=worker_exec, args=(Q, wip_l)).start()
            sleep(0.5)
        
        else: 
            if len(wip_l) == 0:
                print('Finish Draining')
                break 